# Verify Ensemble Submission - No Overlaps

Verify the current submission has no overlapping trees before submitting.

In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely import affinity
from decimal import Decimal
import math

print('Libraries loaded')

In [ ]:
# Tree polygon template
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_polygon(x, y, deg):
    """Get polygon for a tree at position (x, y) with rotation deg."""
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    px = TX * c - TY * s + x
    py = TX * s + TY * c + y
    return Polygon(zip(px, py))

def strip(val):
    """Remove 's' prefix from value."""
    return float(str(val).replace('s', ''))

def has_overlap(trees_data):
    """Check if any trees in the configuration overlap."""
    if len(trees_data) <= 1:
        return False
    
    polys = []
    for _, row in trees_data.iterrows():
        x = strip(row['x'])
        y = strip(row['y'])
        deg = strip(row['deg'])
        polys.append(get_polygon(x, y, deg))
    
    tree_index = STRtree(polys)
    for i, poly in enumerate(polys):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polys[idx]) and not poly.touches(polys[idx]):
                return True
    return False

print('Overlap detection functions defined')

In [ ]:
# Load submission
df = pd.read_csv('/home/submission/submission.csv')
print(f'Loaded {len(df)} rows')

# Check for overlaps in each N
overlap_found = []
for n in range(1, 201):
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    if has_overlap(group):
        overlap_found.append(n)
        print(f'OVERLAP in N={n}')

if overlap_found:
    print(f'\n*** OVERLAPS FOUND in N: {overlap_found} ***')
else:
    print('\n*** NO OVERLAPS - SUBMISSION IS VALID ***')

In [ ]:
# Calculate total score
from numba import njit

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

tx, ty = make_polygon_template()

total_score = 0.0
for n in range(1, 201):
    group = df[df['id'].str.startswith(f'{n:03d}_')]
    xs = np.array([strip(x) for x in group['x']], dtype=np.float64)
    ys = np.array([strip(y) for y in group['y']], dtype=np.float64)
    degs = np.array([strip(d) for d in group['deg']], dtype=np.float64)
    total_score += score_group(xs, ys, degs, tx, ty)

print(f'\nTotal Score: {total_score:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {total_score - 68.922808:.6f}')